In [1]:
import pandas as pd
import numpy as np

#load file
sales = pd.read_csv('sales.csv')
prices = pd.read_csv('prices.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sales.csv'

### Solution #1: Use merge_asof()

In [2]:
#convert ordered_at, updated_at to datetime object (using pd.to_datetime)
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])

In [3]:
sales = sales.sort_values(by='ordered_at')
sales

,product_id,ordered_at,quantity_ordered
86,3954203,2018-09-11 01:43:00,1
28,4085861,2018-09-11 06:26:00,1
26,4085861,2018-09-11 06:53:00,1
27,4085861,2018-09-11 08:24:00,1
123,4085861,2018-09-11 09:30:00,1
...,...,...,...
67,4085861,2018-09-18 20:23:00,1
77,4085861,2018-09-18 20:43:00,1
79,4085861,2018-09-18 20:54:00,1
87,3954203,2018-09-18 21:26:00,1


In [4]:
prices = prices.sort_values(by='updated_at')
prices

,product_id,old_price,new_price,updated_at
4,3954203,68800,60000,2018-09-10 16:32:00
7,3998909,19000,17000,2018-09-10 16:35:00
0,64,270000,239000,2018-09-10 16:37:00
11,4085861,60000,53500,2018-09-11 08:51:00
1,3954203,60000,64000,2018-09-11 11:54:00
9,4085861,53500,67000,2018-09-12 03:51:00
6,3998909,17000,15500,2018-09-13 06:43:00
13,4085861,67000,62500,2018-09-13 06:43:00
3,3954203,64000,60500,2018-09-15 03:49:00
12,4085861,62500,58000,2018-09-15 03:51:00


#### 1.1 With direction='nearest'

In [6]:
# find the "nearest" update timestamp of each order

merge_nearest = pd.merge_asof(sales, prices, left_on='ordered_at', right_on='updated_at', by='product_id', direction='nearest')

In [16]:
# compare ordered_at and "nearest" updated_at of each order to find the correct price

merge_nearest['listed_price'] = np.where(merge_nearest['ordered_at'] >= merge_nearest['updated_at'],
                                         merge_nearest['new_price'], merge_nearest['old_price'])
merge_nearest.head(10)

,product_id,ordered_at,quantity_ordered,price,revenue,old_price,new_price,updated_at,listed_price
0,3954203,2018-09-11 01:43:00,1,64000,64000,68800,60000,2018-09-10 16:32:00,60000
1,4085861,2018-09-11 06:26:00,1,60000,60000,60000,53500,2018-09-11 08:51:00,60000
2,4085861,2018-09-11 06:53:00,1,60000,60000,60000,53500,2018-09-11 08:51:00,60000
3,4085861,2018-09-11 08:24:00,1,60000,60000,60000,53500,2018-09-11 08:51:00,60000
4,4085861,2018-09-11 09:30:00,1,53500,53500,60000,53500,2018-09-11 08:51:00,53500
5,4085861,2018-09-11 11:06:00,1,60000,60000,60000,53500,2018-09-11 08:51:00,53500
6,3954203,2018-09-11 11:11:00,1,60000,60000,60000,64000,2018-09-11 11:54:00,60000
7,3954203,2018-09-11 11:11:00,1,60000,60000,60000,64000,2018-09-11 11:54:00,60000
8,4085861,2018-09-11 11:34:00,1,60000,60000,60000,53500,2018-09-11 08:51:00,53500
9,4085861,2018-09-11 11:47:00,2,60000,120000,60000,53500,2018-09-11 08:51:00,53500


In [17]:
# calculate total revenue for each product

merge_nearest['revenue'] = merge_nearest['quantity_ordered'] * merge_nearest['listed_price']
total_revenue_by_product_and_price = merge_nearest.groupby(['product_id', 'listed_price'], as_index=False)['revenue'].sum()
total_revenue_by_product_and_price

,product_id,listed_price,revenue
0,64,239000,956000
1,3954203,57500,57500
2,3954203,60000,180000
3,3954203,64000,640000
4,3998909,15500,15500
5,3998909,16500,231000
6,3998909,17000,34000
7,4085861,52000,1040000
8,4085861,53500,2140000
9,4085861,58000,2204000


In [18]:
total_revenue = merge_nearest.groupby('product_id')['revenue'].sum()
total_revenue

product_id
64          956000
3954203     877500
3998909     280500
4085861    8247500
Name: revenue, dtype: int64

In [8]:
# Alternative to np.where: create a function with if-else condition

def price(row):
    if row['ordered_at'] >= row['updated_at']:
        return row['new_price']
    else:
        return row['old_price']

merge_nearest['price'] = merge_nearest.apply(price, axis=1)
merge_nearest

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,price
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500
...,...,...,...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,53500,52000,2018-09-17 22:59:00,52000
171,4085861,2018-09-18 20:43:00,1,53500,52000,2018-09-17 22:59:00,52000
172,4085861,2018-09-18 20:54:00,1,53500,52000,2018-09-17 22:59:00,52000
173,3954203,2018-09-18 21:26:00,1,60500,57500,2018-09-17 22:59:00,57500


In [9]:
merge_nearest['revenue1'] = merge_nearest['quantity_ordered'] * merge_nearest['price']
total_revenue1_by_product = merge_nearest.groupby(['product_id'], as_index=False)['revenue1'].sum()
total_revenue1_by_product

,product_id,revenue1
0,64,956000
1,3954203,877500
2,3998909,280500
3,4085861,8247500


#### 1.2. With direction='backward' (combining with direction='forward')

In [28]:
# merge sales and prices table with direction='backward' (default value, which can be optionally omitted)

merge_backward = pd.merge_asof(sales, prices, by='product_id', left_on='ordered_at', right_on='updated_at', direction='backward')
merge_backward = merge_backward[['product_id', 'ordered_at', 'quantity_ordered', 'new_price']].rename(columns={'new_price':'price'})
merge_backward.head(10)

,product_id,ordered_at,quantity_ordered,price
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 06:26:00,1,NaN
2,4085861,2018-09-11 06:53:00,1,NaN
3,4085861,2018-09-11 08:24:00,1,NaN
4,4085861,2018-09-11 09:30:00,1,53500.0
5,4085861,2018-09-11 11:06:00,1,53500.0
6,3954203,2018-09-11 11:11:00,1,60000.0
7,3954203,2018-09-11 11:11:00,1,60000.0
8,4085861,2018-09-11 11:34:00,1,53500.0
9,4085861,2018-09-11 11:47:00,2,53500.0


In [29]:
# merge merge_backward and prices tables with direction='forward'

merge_forward = pd.merge_asof(merge_backward, prices, by='product_id', left_on='ordered_at', right_on='updated_at', direction='forward')
merge_forward.head()

,product_id,ordered_at,quantity_ordered,price,old_price,new_price,updated_at
0,3954203,2018-09-11 01:43:00,1,60000.0,60000.0,64000.0,2018-09-11 11:54:00
1,4085861,2018-09-11 06:26:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
2,4085861,2018-09-11 06:53:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
3,4085861,2018-09-11 08:24:00,1,NaN,60000.0,53500.0,2018-09-11 08:51:00
4,4085861,2018-09-11 09:30:00,1,53500.0,53500.0,67000.0,2018-09-12 03:51:00


In [30]:
# fill null values in 'price' column by 'old_price' column where necessary

merge_forward['price'] = merge_forward['price'].fillna(merge_forward['old_price'])
final_data = merge_forward[['product_id', 'ordered_at', 'quantity_ordered', 'price']]
final_data

,product_id,ordered_at,quantity_ordered,price
0,3954203,2018-09-11 01:43:00,1,60000.0
1,4085861,2018-09-11 06:26:00,1,60000.0
2,4085861,2018-09-11 06:53:00,1,60000.0
3,4085861,2018-09-11 08:24:00,1,60000.0
4,4085861,2018-09-11 09:30:00,1,53500.0
...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,52000.0
171,4085861,2018-09-18 20:43:00,1,52000.0
172,4085861,2018-09-18 20:54:00,1,52000.0
173,3954203,2018-09-18 21:26:00,1,57500.0


In [31]:
final_data['revenue'] = final_data['quantity_ordered'] * final_data['price']
revenue_by_product_and_price = final_data.groupby(['product_id', 'price'], as_index=False)['revenue'].sum()
revenue_by_product_and_price

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_9020\2432931725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['revenue'] = final_data['quantity_ordered'] * final_data['price']


,product_id,price,revenue
0,64,239000.0,956000.0
1,3954203,57500.0,57500.0
2,3954203,60000.0,180000.0
3,3954203,64000.0,640000.0
4,3998909,15500.0,15500.0
5,3998909,16500.0,231000.0
6,3998909,17000.0,34000.0
7,4085861,52000.0,1040000.0
8,4085861,53500.0,2140000.0
9,4085861,58000.0,2204000.0


In [32]:
total_revenue = final_data.groupby('product_id', as_index=False)['revenue'].sum()
total_revenue

,product_id,revenue
0,64,956000.0
1,3954203,877500.0
2,3998909,280500.0
3,4085861,8247500.0


### Solution #2: Use def function

In [3]:
#def function that return the closest date that price update
def closest_update_date (id, order_date):
    new_date = prices['updated_at'][prices['product_id'] == id]
    if order_date < new_date.min():
       closest_date = new_date.min()
    else :
       closest_date = new_date[order_date > new_date].max()                            
    return closest_date

In [9]:
#def function that return closest price base on productid and order_date
def return_closest_price(id, order_date):
    day = closest_update_date (id, order_date)
    if order_date < day :
       closest_price = prices['old_price'][(prices['updated_at'] == day) & (prices['product_id'] == id)].values[0]
    else :
       closest_price = prices['new_price'][(prices['updated_at'] == day) & (prices['product_id'] == id)].values[0]                              
    return closest_price

In [10]:
#Find price corresponding each row and append to update_price array.
update_price=[]
for i in np.arange(len(sales)):
    updated_price = return_closest_price(sales.iloc[i]['product_id'], sales.iloc[i]['ordered_at'])
    update_price.append(updated_price)

#create and calculate price and revenue column
sales["price"] = update_price
sales["revenue"] = sales['price'] * sales['quantity_ordered']

#export to csv file
sales
# sale.to_csv("revenue.csv")

,product_id,ordered_at,quantity_ordered,price,revenue
0,3998909,2018-09-18 17:51:00,1,16500,16500
1,3998909,2018-09-18 12:52:00,1,16500,16500
2,3998909,2018-09-18 11:33:00,1,16500,16500
3,3998909,2018-09-18 18:47:00,1,16500,16500
4,3998909,2018-09-18 17:36:00,1,16500,16500
...,...,...,...,...,...
170,4085861,2018-09-13 23:29:00,1,62500,62500
171,64,2018-09-11 19:32:00,1,239000,239000
172,64,2018-09-14 20:10:00,1,239000,239000
173,64,2018-09-12 16:00:00,1,239000,239000


In [22]:
sales.groupby('product_id').agg({'revenue':sum})

,revenue
product_id,
64,956000
3954203,877500
3998909,280500
4085861,8247500
